Project by Maged Own
<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium==0.5.0
import folium # map rendering library

! pip install requests

! pip install beautifulsoup4
! pip install lxml

print('Libraries imported.')

     |████████████████████████████████| 112kB 6.9MB/s eta 0:00:01
     |████████████████████████████████| 122kB 4.3MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 5.8MB/s eta 0:00:011     |██████████████████████████████  | 5.2MB 12.5MB/s eta 0:00:01
Libraries imported.


The code above identifies the librarys that need importing to execute this project. 

In [57]:
#from bs4 import BeautifulSoup

#website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#soup = BeautifulSoup(website_url,'lxml')

response = requests.get(url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",)

print(response.status_code)

# define the dataframe columns

# instantiate the dataframe
neighborhoods = pd.DataFrame()

200


In [12]:

url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(url,'lxml')
Toronto_table = soup.find("table", {'class':'wikitable sortable'})
tbody = Toronto_table.find('tbody')
trs = tbody.find_all('tr')
Toronto = []
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        Toronto.append(td.text)

converted_list = []
for element in Toronto:
    converted_list.append(element.strip())

data_iter = iter(converted_list)
New_table = zip(*[data_iter]*3)
neighborhoods = pd.DataFrame(New_table)
new_columns = ['Postal Code','Borough','Neighborhood']
neighborhoods.columns = new_columns

Now that we transferred the information from wikipedia. We can modify the data so that it serves our functions.

In [4]:
# the below is to drop neighborhoods without assigned Boroughs
neighborhoods = neighborhoods[neighborhoods.Borough != 'Not assigned']
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    ))

The dataframe has 10 boroughs and 103 neighborhoods.


In [5]:
#checking for Neighborhood which has a value of "not assigned"
neighborhoods[neighborhoods.Neighborhood == 'Not assigned']

,Postal Code,Borough,Neighborhood


This means that we don't have any values for neighborhood that are not assigned.

In [6]:
temp_neighborhoods = neighborhoods.groupby('Postal Code')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_neighborhoods

df_merge = pd.merge(neighborhoods, temp_neighborhoods, on = "Postal Code")

df_merge.drop(['Neighborhood_x'], axis = 1, inplace = True)
new_columns = ['Postal Code','Borough','Neighborhood']
df_merge.columns = new_columns
df_merge.head()


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


The above shows the unique values of postal codes and their neighborhoods

In [7]:
df_merge.shape

(103, 3)

The above shows that we now have 103 rows. The above finishes task one for this project.

In [66]:
r = requests.get('http://cocl.us/Geospatial_data').text
soup2 = BeautifulSoup(r,'lxml')

ps = soup2.find_all('p')
data = []
for x in ps:
    data.append(str(x))
newlist = [word for line in data for word in line.split()]

geo_data=pd.DataFrame(newlist)

geo_data = geo_data.drop([0,1]).reset_index()
geo_data = geo_data.drop(['index'],axis=1)
geo_data.head()

geo_data = pd.concat([geo_data[0].str.split(',', expand=True)], axis=1)
new_columns2 = ['Postal Code','Latitude','Longitude']
geo_data.columns = new_columns2
geo_df = geo_data


,Postal Code,Latitude,Longitude
0,M1B,43.8066863,-79.1943534
1,M1C,43.7845351,-79.1604971
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.2169174
4,M1H,43.773136,-79.2394761


In [67]:
geo_merged = pd.merge(geo_df, df_merge, on='Postal Code')
geo_data=geo_merged[['Postal Code','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8066863,-79.1943534
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761


In [69]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.6763574,-79.2930312
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6795571,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.6689985,-79.3155716
43,M4M,East Toronto,Studio District,43.6595255,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.7280205,-79.3887901


In [70]:
CLIENT_ID = 'MGICIYLWVCZ0L45PW42XQV1XLRRFQFIHBRBEJND1DGCVK32G' # your Foursquare ID
CLIENT_SECRET = 'AMVLK1LEOWZBCQT2431IC20UCRT1AKC5WMNF5PV3G12OMGGY' # your Foursquare Secret
VERSION = '20180604'

In [71]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
         #create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
         #make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [72]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [95]:
toronto_venues.groupby('Neighborhood').count()
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()


In [96]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)



In [98]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

In [99]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.6763574,-79.2930312,1,Asian Restaurant,Health Food Store,Trail,Pub,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6795571,-79.352188,4,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Spa,Japanese Restaurant,Juice Bar,Brewery
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.6689985,-79.3155716,4,Park,Fast Food Restaurant,Gym,Brewery,Sandwich Place,Restaurant,Pub,Coffee Shop,Pizza Place,Movie Theater
43,M4M,East Toronto,Studio District,43.6595255,-79.340923,4,Coffee Shop,Brewery,Gastropub,Café,American Restaurant,Bakery,Yoga Studio,Convenience Store,Seafood Restaurant,Cheese Shop
44,M4N,Central Toronto,Lawrence Park,43.7280205,-79.3887901,1,Park,Bus Line,Business Service,Swim School,Yoga Studio,Discount Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


In [100]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="T_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [106]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=(0.7).add_to(map_clusters)
map_clusters       

SyntaxError: invalid syntax (<ipython-input-106-f9e95ccbb2f9>, line 21)